## AI-Powered Resume Analyzer & Job Matcher


#### 1. Imports & Environment Setup


In [1]:
# 1. Imports
from dotenv import load_dotenv
from typing import List

# LangChain Imports
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from pydantic import BaseModel, Field

# Load Environment Variables (API Key)
load_dotenv()

# Initialize the LLM (We use gpt-4o-mini for speed and cost)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

print("✅ Environment and LLM initialized.")

/Users/yashjain/Developer/youtube/langchain-youtube/.langchain-youtube-env/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


✅ Environment and LLM initialized.


#### 2. Defining Data Models (Structured Output)


In [2]:
# 2. Define Structured Output Models


class ResumeAnalysis(BaseModel):
    skills: List[str] = Field(description="List of technical and soft skills found")
    experience_years: int = Field(description="Total years of experience")
    strengths: List[str] = Field(description="Key strengths of the candidate")
    weaknesses: List[str] = Field(description="Areas for improvement")
    ats_score: int = Field(description="ATS compatibility score out of 100")


class JobMatch(BaseModel):
    match_percentage: int = Field(description="Match percentage with job description")
    matching_skills: List[str] = Field(description="Skills that match the job")
    missing_skills: List[str] = Field(description="Skills required but missing")
    recommendations: List[str] = Field(description="Recommendations to improve match")


print("✅ Data Models defined.")

✅ Data Models defined.


#### 3. The Resume Analyzer Chain


In [3]:
# 3. Create Resume Analyzer Chain

# Define the Prompt
analyzer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert resume analyst. Analyze the resume and provide detailed feedback."),
        ("human", "Resume Content:\n{resume}"),
    ]
)

# Connect Prompt to LLM with Structured Output
# This forces the LLM to return the Pydantic object, not just text
analyzer_chain = analyzer_prompt | llm.with_structured_output(ResumeAnalysis)

# --- DEMO DATA ---
sample_resume = """
Yash Jain - AI Software Engineer
Experience:
- 6 years as AI Developer at Tech Corp
- Built REST APIs using Python and Flask
- Managed PostgreSQL databases
Skills: Python, LangChain, LangGrapg, Flask, FastAPI, Docker, SQL, Git
Education: BE in Computer Science, 2020
"""

# Run the chain
analysis_result: ResumeAnalysis = analyzer_chain.invoke({"resume": sample_resume})

# Display Results
print(f"📊 ATS Score: {analysis_result.ats_score}")
print(f"✅ Skills: {analysis_result.skills}")
print(f"⚠️ Weaknesses: {analysis_result.weaknesses}")

📊 ATS Score: 85
✅ Skills: ['Python', 'LangChain', 'LangGraph', 'Flask', 'FastAPI', 'Docker', 'SQL', 'Git']
⚠️ Weaknesses: ['Lack of specific project details', 'No mention of soft skills or teamwork experience', 'Education is recent and may lack advanced degrees or certifications']


#### 4. The Job Matcher Chain


In [4]:
# 4. Create Job Matching Chain

matcher_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert recruiter. Compare the resume with the job description."),
        ("human", "Resume: {resume}\nJob Description: {job_description}"),
    ]
)

matcher_chain = matcher_prompt | llm.with_structured_output(JobMatch)

# --- DEMO DATA ---
job_description = """
Senior AI Developer
Requirements:
- 5+ years of experience in AI development
- Strong Python and FastAPI experience
- Database: SQL, MongoDB
- Cloud: AWS, Docker, Kubernetes
"""

# Run the chain
match_result: JobMatch = matcher_chain.invoke({"resume": sample_resume, "job_description": job_description})

# Display Results
print(f"📈 Match Percentage: {match_result.match_percentage}%")
print(f"❌ Missing Skills: {match_result.missing_skills}")
print(f"💡 Recommendations: {match_result.recommendations}")

📈 Match Percentage: 75%
❌ Missing Skills: ['FastAPI', 'MongoDB', 'AWS', 'Kubernetes']
💡 Recommendations: ['Gain experience with FastAPI to match the job requirements.', 'Learn MongoDB to enhance database skills.', 'Familiarize yourself with AWS and Kubernetes for cloud experience.']


#### 5. The Resume Improver


In [5]:
# 5. Resume Improvement Chain

improver_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a professional resume writer.
            Provide 3 specific bullet points to improve the resume based on the analysis.""",
        ),
        ("human", "Original Resume: {resume}\nAnalysis: {analysis}"),
    ]
)

# Simple StrOutputParser because we just want text back
improver_chain = improver_prompt | llm | StrOutputParser()

# Run the chain
suggestions = improver_chain.invoke(
    {
        "resume": sample_resume,
        "analysis": str(analysis_result),
    }  # Pass the previous a nalysis result
)

print("📝 Improvement Suggestions:\n")
print(suggestions)

📝 Improvement Suggestions:

To enhance Yash Jain's resume and improve its effectiveness, consider the following bullet points:

1. **Detail Specific Projects and Achievements**: 
   - "Led the development of a machine learning model that improved predictive accuracy by 30%, resulting in enhanced decision-making processes for clients."
   - "Designed and implemented a REST API for a high-traffic application, achieving a response time reduction of 40% through optimized code and efficient database queries."

2. **Highlight Soft Skills and Teamwork**: 
   - "Collaborated with cross-functional teams to integrate AI solutions into existing products, fostering a culture of innovation and continuous improvement."
   - "Mentored junior developers in AI best practices and coding standards, enhancing team productivity and knowledge sharing."

3. **Include Relevant Certifications or Continuing Education**: 
   - "Pursuing advanced certifications in AI and machine learning, such as TensorFlow Devel

#### 6. Interactive Career Coach (Memory)


In [6]:
# 6. Interactive Career Coach with Memory

# Memory Storage Dictionary
store = {}


def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


# Define Chat Prompt with History Placeholder
coach_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a career coach. You remember the candidate's resume details. Be encouraging."),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

# Create the chain
coach_chain = coach_prompt | llm | StrOutputParser()

# Wrap chain with Message History
coach_with_memory = RunnableWithMessageHistory(
    coach_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

# --- INTERACTIVE DEMO ---
session_id = "user_123"

# 1. Seed the memory with context
initial_context = f"I have these skills: {analysis_result.skills}. My match score is {match_result.match_percentage}%."
coach_with_memory.invoke(
    {"input": initial_context},
    config={
        "configurable": {"session_id": session_id},
    },
)
print("🤖 Coach: Context received. How can I help?")

# 2. Ask follow-up questions
questions = [
    "What is the one skill I should learn next?",
    "How do I explain my missing experience in an interview?",
]

for q in questions:
    print(f"👤 User: {q}")
    response = coach_with_memory.invoke(
        {"input": q},
        config={
            "configurable": {"session_id": session_id},
        },
    )
    print(f"🤖 Coach: {response}")

/Users/yashjain/Developer/youtube/langchain-youtube/.langchain-youtube-env/lib/python3.14/site-packages/pydantic/v1/main.py:1054: UserWarning: LangSmith now uses UUID v7 for run and trace identifiers. This warning appears when passing custom IDs. Please use: from langsmith import uuid7
            id = uuid7()
Future versions will require UUID v7.
  input_data = validator(cls_, input_data)


🤖 Coach: Context received. How can I help?
👤 User: What is the one skill I should learn next?
🤖 Coach: Given your strong foundation in Python and web frameworks like Flask and FastAPI, a great skill to consider learning next is **machine learning** or **data science**. This would complement your existing skills and open up new opportunities, especially with your knowledge of SQL for data manipulation.

You could start with libraries like **scikit-learn** or **TensorFlow**. Understanding the basics of machine learning algorithms and how to implement them can significantly enhance your profile, especially if you're interested in roles that involve AI or data analysis.

Additionally, if you're leaning towards web development, exploring **JavaScript** and frameworks like **React** could also be beneficial, as they are highly sought after in full-stack development roles.

Whichever path you choose, remember that continuous learning is key, and you're already on a great trajectory! Keep up t